In [10]:
import xarray as xr
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import math
import calendar as clndr
import glob
from scipy.interpolate import griddata
from matplotlib import interactive
from matplotlib import gridspec
from matplotlib import rcParams
from parcels import FieldSet, ParticleSet, Variable, JITParticle, AdvectionRK4, plotTrajectoriesFile
from datetime import timedelta
from operator import attrgetter

In [11]:
# -----------------------------------------------------------------------------
# Input variables, change these
# -----------------------------------------------------------------------------

# year want
yr_wnt = 1993

# set begin month and number of months to track particles over
# 1) num_mon_track < 0, run parcels backward
# 2) num_mon_track > 0, run parcels forward
mon_start = 1
num_mon_track = 11

# use geo or total
# 1) val_wnt = geo, use geostrophic velocity
# 2) val_wnt = total, use total (geostrophic + ekman)
# vel_wnt = 'geo'
vel_wnt = 'total'

# directory of ccs particles files (Note: you will need to change this to your directory)
dir_particles_ccs = '/Users/esaraf/Desktop/PRC/CCS'

# box size
# 1) lat_box is in degrees, monterey bay: lat_box=[36, 38]; baja: lat_box=[28.5, 30.5]
# 2) lon_box is in grid points next to shore, ie [1,2] the 1 is the grid cell next to land,
#    the 2 is located two grid cells from land
lon_box = [1, 4]
lat_box = [36, 38]
# lat_box = [28.75, 30.25]

# we want more particles to release, interpolate using these grid spacings
dx = 0.5
dy = 0.5

# name of output netcdf created by the particles run
dir_particles = '/Users/esaraf/Desktop/PRC/Released_total'

# the output netcdf doesn't save all time steps, set the number wanted in hours
hrs_to_save = 24

# run the particles backward in time
bck_td = timedelta(minutes=5)

# lat and lon limits for plot
lon_bgn = -150
lon_end = -110
lat_bgn = 20
lat_end = 50

# name of the output plots (NOTE: you will need to make this directory)
dir_plots = '/Users/esaraf/Desktop/PRC/Plots'

# -----------------------------------------------------------------------------
# END: Input variables, change these
# -----------------------------------------------------------------------------


In [12]:
# 1) Get file list
# use numpy datetime64 timedelta to find month range
mon1 = np.datetime64('{}-{:02d}'.format(yr_wnt, mon_start), 'M')
mon2 = mon1 + np.timedelta64(num_mon_track, 'M')

# sort the months so that they occur in chronological order
mon_rng = np.sort([mon1, mon2])

# set begin and end mons
in_min = np.argmin(mon_rng)
in_max = np.argmax(mon_rng)
mon_bgn = mon_rng[in_min]

# final dates of the particles to track, need to add 1 month to the last month to get
# all days of the final month
datesD = np.arange(mon_rng[0], mon_rng[1] + np.timedelta64(1, 'M'), dtype='datetime64[D]')
num_datesD = len(datesD)

# get a list of all CCS files that correspond to datesD
file_list = list()
for i in range(num_datesD):
    # date to get file
    datesDi = datesD[i]

    # year
    yri = datesDi.astype('str')[0:4]

    # get 'day-of-year' of datesDi
    diff_jan1 = datesDi - np.datetime64('{}-01-01'.format(yri))
    doy = diff_jan1.astype('int') + 1

    # filename corresponding to this doy
    dir_yr = '{}/{}'.format(dir_particles_ccs, yri)
    fn_yr_doy = '{}/{}_{:03d}_{}_uv.nc'.format(dir_yr, yri, doy, vel_wnt)

    # append to the file list
    file_list.append(fn_yr_doy)

# run the particles over all files
days_to_run = num_datesD - 1


In [13]:
# 2) Initial paritle release positions
# open sample file to get location of release politions
ds1_smpl = xr.open_dataset(file_list[0])

# narrow dataset for the lat range of the box, just use the U value as we are just
# interested in data/land values (land values = np.nan)
da1_smpl = ds1_smpl['U'][0, :, :].sel(lat=slice(lat_box[0], lat_box[1]))

# get rid of inland data such as Gulf of CA
for i in range(len(da1_smpl.lat)):
    datai = da1_smpl.data[i, :]
    in_miss = np.isnan(datai).nonzero()[0]
    da1_smpl[i, in_miss[0]:] = np.nan


# size of the box
ny_box = da1_smpl.shape[0]
nx_box = lon_box[1] - lon_box[0] + 1

# loop over all lats in the box and get the x grid cells
lat_points_mtrx = np.zeros([ny_box, nx_box])
lon_points_mtrx = np.zeros([ny_box, nx_box])
for i in range(ny_box):
    da1i = da1_smpl[i, :]
    in_wtr = np.isfinite(da1i)
    da1f = da1i[in_wtr]
    nxf = da1f.shape[0]
    nx_end = nxf - lon_box[0] 
    nx_bgn = nxf - lon_box[1]
    lat_points_mtrx[i, :] = da1f['lat'].data
    lon_points_mtrx[i, :] = da1f['lon'].data[nx_bgn:nx_end+1]

# these are the points in the box associated with the Globcurrent 0.25 deg grid
lat_points = np.reshape(lat_points_mtrx, [ny_box*nx_box])
lon_points = np.reshape(lon_points_mtrx, [ny_box*nx_box])

# we want more particles to release, use griddata to interpolate extra points in the box
# first set up new interpolation grid with the requested grid spacing
long, latg = np.meshgrid(np.arange(np.min(lon_points), np.max(lon_points)+dx, dx),
                         np.arange(np.min(lat_points), np.max(lat_points)+dx, dx))

# griddata using linear interpolation
points = (lon_points, lat_points)
values = np.ones(len(lon_points))  # fake values, we will use linear interpolation which will have NaN outside 
grid_x = long
grid_y = latg
grid_z2 = griddata(points, values, (grid_x, grid_y), method='linear')

# we are not interested in the fake data, we just need the lon-lat values, but set NaN of the fake data 
# to NaN in the lon and lat grid
inm_grid_z2 = np.isnan(grid_z2)
long[inm_grid_z2] = np.nan
latg[inm_grid_z2] = np.nan

# reshape lon and lat as vectors and remove NaN for final release positions
nyg, nxg = long.shape
lon_pos_re = np.reshape(long, [nyg*nxg])
lat_pos_re = np.reshape(latg, [nyg*nxg])
ind_pos_re = np.isfinite(lon_pos_re)
lon_pos = lon_pos_re[ind_pos_re]
lat_pos = lat_pos_re[ind_pos_re]


In [14]:
# 3) Plot the release points
lon_glob_grid, lat_glob_grid = np.meshgrid(da1_smpl.lon, da1_smpl.lat)
plt.contourf(da1_smpl.lon,da1_smpl.lat, da1_smpl.data)
plt.plot(lon_glob_grid, lat_glob_grid, '.k')
plt.plot(lon_pos, lat_pos, 'pr')

plt.xlim([np.min(lon_pos)-0.5, np.max(lon_pos)+0.5])
plt.ylim([np.min(lat_pos)-0.5, np.max(lat_pos)+0.5])

plt.title('Red stars=release position, black circles=grid points')

# some points are on "land", this doesn't matter in the Parcels code

Text(0.5, 1.0, 'Red stars=release position, black circles=grid points')

In [15]:
# 4) Put data into Paricles
# tell particles the location and name of the data wanted
filenames = {'U': file_list, 'V': file_list}

# The first element of the pair is the name particles uses, the second is the name used in the
# netcdf files that we created and saved on the Desktop
variables = {'U': 'U',
             'V': 'V'}
dimensions = {'lat': 'lat',
              'lon': 'lon',
              'time': 'time'}

# Read the netcdf files and get them into particles
fieldset = FieldSet.from_netcdf(filenames, variables, dimensions)

In [16]:
# 5) Initialize release points in box
pset = ParticleSet.from_list(fieldset=fieldset,   # the fields on which the particles are advected
                             pclass=JITParticle,  # the type of particles (JITParticle or ScipyPartile)
                             lon=lon_pos, # a vector of release longitudes 
                             lat=lat_pos)    # a vector of release latitudes

In [17]:
# 6) Create output file and run particles
# output file name, with the number of time intervals to be saved in hours

file_particles = '{}/{}_ccs_particles_Yr_{}_Mon_{}_{}_Box_{}_{}_{}_{}.nc'.format(dir_particles, vel_wnt, yr_wnt, mon_start, num_mon_track, lat_box[0], lat_box[1], lon_box[0], lon_box[1])
output_file = pset.ParticleFile(name=file_particles, outputdt=timedelta(hours=hrs_to_save))

# run the particles (back_or_frwrd=1 then run forward, back_or_frwrd=-1 then run backward)
back_or_frwrd = 1
if num_mon_track < 0:
    back_or_frwrd = -1

pset.execute(AdvectionRK4,
             runtime=timedelta(days=days_to_run),
             dt=bck_td*back_or_frwrd,
             output_file=output_file)

# save the paricle tracks to the output netcdf file
output_file.export()


INFO: Compiled JITParticleAdvectionRK4 ==> /var/folders/83/kh6bxj8x5x3b6ht9ttlh9b140000gp/T/parcels-502/c73cfdcdf1822f06bdd84a4deabf6652.so


PermissionError: [Errno 13] Permission denied: b'/Users/esaraf/Desktop/PRC/Released_total/total_ccs_particles_Yr_1993_Mon_1_11_Box_36_38_1_4.nc'

In [ ]:
# 7) Plot particle tracks

# plot paramaters
params = {
    'savefig.dpi': 300,  # to adjust notebook inline plot size
    'font.size': 5,  
    'legend.fontsize': 5, 
    'xtick.labelsize': 5,
    'ytick.labelsize': 5,
    'figure.figsize': [5, 3],
    'font.family': 'STIXGeneral',
    'savefig.bbox': 'tight'
}
rcParams.update(params)

# plot particles
ds1 = xr.open_dataset(file_particles)
var_ds1 = list(ds1.keys())
coord_ds1 = list(ds1.coords.keys())


cmap = mpl.cm.get_cmap('Spectral')
if back_or_frwrd == -1:
    cmap = mpl.cm.get_cmap('Spectral')
rgba = cmap(0.5)

lat_prt = ds1['lat'].data
lon_prt = ds1['lon'].data
time_prt = ds1['time'].data

num_particles, num_dates = lat_prt.shape
fract_dates = np.linspace(0, 1, num_dates)

# fig, ax = plt.subplots()


# create new figure and close old one
plt.close()
fig = plt.figure()

# subplots using gridspec
num_row = 1
num_clmn = 1
gs1 = gridspec.GridSpec(num_row, num_clmn)
gs1.update(left=0.1, right=0.9, bottom=0.1,
           top=0.9, wspace=0.1, hspace=0.1)

ax1 = plt.subplot(gs1[0, 0], projection=ccrs.PlateCarree())

# coastlines
land1 = cfeature.NaturalEarthFeature(
    'physical', 'land', '50m', edgecolor='lightgray',
    facecolor='lightgray', zorder=100, linewidth=0.5)

# land mask
ax1.add_feature(land1)

# plot particle tracks
for i in range(num_particles):
    plt.scatter(lon_prt[i, :], lat_prt[i, :], c=fract_dates,
                vmin=np.min(fract_dates), vmax=np.max(fract_dates),
                s=.5, cmap=cmap, marker=".", zorder=200)

# set lon and lat limits
plt.xlim([lon_bgn, lon_end])
plt.ylim([lat_bgn, lat_end])

# colorbar
in_tcks_cbar = np.linspace(0, 1, 5)
in_tcks_time = np.linspace(0, num_dates-1, 5).astype('int')
cbar = plt.colorbar(ticks=in_tcks_cbar)
cbar.ax.set_yticklabels(time_prt[0, in_tcks_time])  # vertically oriented colorbar

# add fancy title, including month name
mon_start_lbl = clndr.month_name[mon_start]

back_frwrd_lbl = 'forward'
if back_or_frwrd == -1:
    back_frwrd_lbl = 'backward'

if vel_wnt=='geo':
    vel_wnt_lbl = 'geostrophic'
else:
    vel_wnt_lbl = 'total (Ekman + geostrophic)'
ttl_lbl = '{} {} {} run over {} months\n{} velocities\nRelease box: {}-{}$^\circ$N, {}-{} grid points from shore'.format(mon_start_lbl, yr_wnt, back_frwrd_lbl, np.abs(num_mon_track), vel_wnt_lbl, lat_box[0], lat_box[1], lon_box[0], lon_box[1])
plt.title(ttl_lbl)

# save figure (NOTE: you will need to make the output directory)
file_plot = '{}/{}_ccs_particles_Yr_{}_Mon_{}_{}_Box_{}_{}_{}_{}.png'.format(dir_plots, vel_wnt, yr_wnt, mon_start, num_mon_track, lat_box[0], lat_box[1], lon_box[0], lon_box[1])

plt.savefig(file_plot)

In [ ]:
print(ttl_lbl)

In [ ]:
open('/Users/esaraf/Desktop/NOAA/Particles_ccs_files_total/2002/2002_336_total_uv.nc')